In [1]:
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

https://openaccess.uoc.edu/bitstream/10609/148363/3/miravecaTFG0623memoria.pdf

https://www.portalinmobiliario.com/robots.txt


SAAS
https://github.com/Saas-Starter-Kit/Saas-Kit-prisma?tab=readme-ov-file


In [3]:
from webscrapper_portal_inmobiliario import WebScraperPortalInmobiliario

### TODO:
- ~~trackear propiedades en el tiempo para el mismo sector  SE HACE CON LA [LAT LONG]~~
- ~~Sortear baneos de ip por muchas request - sectores con muchas propiedades~~
https://brightdata.com/cp/zones/residential_proxy1/edit 8 dolares el GB --> SOLUCION
- ~~crear tabla intetmedia con 3 tablas y generar id unico...~~
- ~~buscar estandar para nombres de columnas en tablas, en ingles... sin mayusculas~~
- Hacer que se descarguen las cards urls totales [venta + arriendo]
- Los polygonos se agregan a una tabla en la base de datos... con una funcion
- para correr el programa se ejecuta otra funcion que leera los polygoinos de la base de datos
- ejecutar todos los links en tread
---
- Calculos de estadicticas de rentabilidad
- rankear propiedades basado en esto
- grafico mapa donde se encuentran estas propiedades
- extraer imagenes del TOP 3 Propiedades
- plotear en single plot matplotlib. una especie de reporte automatico.

### FIXES
- ~~Cambiar id unico al titulo, se pierden precios con lat long porque poseen la misma ubicacion~~
- ~~hacer que cuando falle, re llenenen con nan los valores faltantes~~
- ~~al cambiar en total number of request max--> 0, se pierde el hilo para sacar la tabla fallando~~
- ~~las tablas no estan siendo detectadas por xpath ni en soup tables.. aparentemente quedan en un formato lazy,
   pero la data esta ahi en un diccionario~~
   - ~~El problema sucedia porque se requere un wait para que carge todo... esto afecta aveces tambien a las paginas main urls.. se nesesita aplicar~~
- ~~El nuevo formato de pagina low/webscrapping tiene todo lo nesesario y la full cambia en nombre al texto de "UBICACION"~~

-  ~~se presentan errores en determinadas propiedades con la nueva metodologia de extraccioon de tabla. pero aveces no sucede. trackear error~~


### APRENDIZAJES

webdriver
2. es bueno reiniciar y cerrar el driver
2. el estado de la pagina (restricciones) cambiara solo con el refresh!!
3. Si se aplica un init y luego get/request no retornara NADA (PUEDE QUE SI PERO EN GENERAL NO PORQUE ES UN USER NUEVO SE ASOCIARA CON WEBSCRAPPING)
4. el orden correcto sera

close--> init--> request (primera vez ,  alta probabilida de falla) -->  refresh (recarga y se puede obtener la tabla y ubicaciion)

In [4]:
WSPI = WebScraperPortalInmobiliario(tipo_operacion="venta",
                                    tipo_inmueble="departamento",
                                    theme="default",
                                    folder_save_name="diego_portales",
                                    debug=False)

proxy enabled!, random session id:  0.7295693959103857


  0%|          | 0/100 [00:00<?, ?it/s]

*** total number of properties: 
 arriendo 37
 venta 44
*** url: https://www.portalinmobiliario.com/arriendo/departamento/_DisplayType_M_item*location_lat:-33.565285*-33.551624,lon:-70.56922*-70.543556#1
------------------------
restarting webdriver because xpath not found
*** url: https://www.portalinmobiliario.com/venta/departamento/_DisplayType_M_item*location_lat:-33.565285*-33.551624,lon:-70.56922*-70.543556#1
------------------------
master_id:  7
master_id:  7


In [9]:
WSPI.compile_results_df_to_db()

master_id:  7
master_id:  7
master_id:  8
master_id:  8
master_id:  9
master_id:  9
master_id:  10
master_id:  10
master_id:  11
master_id:  11
master_id:  12
master_id:  12
master_id:  13
master_id:  13
master_id:  14
master_id:  14
master_id:  15
master_id:  15
master_id:  16
master_id:  16
master_id:  17
master_id:  17
master_id:  17
master_id:  17
master_id:  18
master_id:  18
master_id:  19
master_id:  19
master_id:  17
master_id:  17
master_id:  20
master_id:  20
master_id:  17
master_id:  17
master_id:  20
master_id:  20
master_id:  17
master_id:  17
master_id:  17
master_id:  17
master_id:  21
master_id:  21
master_id:  22
master_id:  22
master_id:  23
master_id:  23
master_id:  24
master_id:  24
master_id:  25
master_id:  25
master_id:  26
master_id:  26
master_id:  27
master_id:  27
master_id:  28
master_id:  28
master_id:  29
master_id:  29
master_id:  30
master_id:  30
master_id:  31
master_id:  31
master_id:  32
master_id:  32
master_id:  33
master_id:  33
master_id:  34
m

In [6]:
WSPI.dict_len_type_operations

{'arriendo': 36, 'venta': 40}

In [11]:
["arriendo"]*WSPI.dict_len_type_operations["arriendo"]+["venta"]*WSPI.dict_len_type_operations["venta"]

['arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'arriendo',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta',
 'venta']

LOAD MAPS FROM DB

In [33]:
np.ceil(np.asarray([0,2]) / 50).astype(int)

array([0, 1])

In [21]:
from ipyleaflet import Map, DrawControl, TileLayer, GeoJSON

main_interactive_map = WSPI.init_map_ipyflet(theme=WSPI.theme,
                                             center_map_cordinates=WSPI.center_map_coordinates,
                                             geojson_data=WSPI.picked_pts_features)

draw_control = DrawControl()

draw_control.on_draw(WSPI.handle_draw)

main_interactive_map.add(draw_control)
display(main_interactive_map)  # NOQA

Map(center=[-33.4489, -70.6693], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

#### benchamarks

1145 PROPIEDADES ESCRAPIADAS EN 56 MINUTOS -  SINGLE THREAD

# DATA MANAGE
------------------------------------------

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from database import *

In [7]:
conn = create_conect_db("real_state.db")
df = get_joined_data_as_dataframe(conn=conn, threshold_date="2024-07-24 15:00")
df = df.T.drop_duplicates().T
df

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,orientacion,gastos_comunes,titulo,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date
0,-33.405653,-70.560099,2.0,2.0,2.0,188.0,68.0,2.0,1.0,7.0,...,Sur,180000.0,Con Jardín: Nuestra Señora Del Rosario / Estad...,https://www.portalinmobiliario.com/MLC-1519331...,metro_manquehue,0,1321110.0,35.0,arriendo,2024-08-30 10:19:34.222238
1,-33.410475,-70.5725,2.0,4.0,3.0,130.0,124.0,2.0,1.0,31.0,...,Nor-Oriente,0.0,Departamento En Arriendo De 4 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-1519330...,metro_manquehue,0,1509840.0,40.0,arriendo,2024-08-30 10:19:34.229592
2,-33.401758,-70.558145,23.0,2.0,2.0,70.0,63.0,0.0,1.0,0.0,...,Nor-Poniente,190000.0,Espectacular Departamento Tipo Mariposa En Las...,https://www.portalinmobiliario.com/MLC-1511659...,metro_manquehue,0,1019142.0,27.0,arriendo,2024-08-30 10:19:34.237387
3,-33.412626,-70.577014,23.0,1.0,1.0,38.0,38.0,1.0,1.0,4.0,...,Poniente,115000.0,Departamento Arriendo De 1 Dorm. Estadio Españ...,https://www.portalinmobiliario.com/MLC-2643503...,metro_manquehue,0,668859.12,17.72,arriendo,2024-08-30 10:19:34.245242
4,-33.411946,-70.566881,22.0,3.0,2.0,90.0,80.0,1.0,1.0,10.0,...,Poniente,240000.0,"Depto En Arriendo De 3 Dorm. En Las Condes, Me...",https://www.portalinmobiliario.com/MLC-1512054...,metro_manquehue,0,1050000.0,27.817517,arriendo,2024-08-30 10:19:34.252004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,-33.410506,-70.573783,7.0,4.0,3.0,115.0,115.0,1.0,1.0,45.0,...,NO,220000.0,Dpto Remodelado Luminoso,https://www.portalinmobiliario.com/MLC-2670120...,metro_manquehue,1,320841000.0,8500.0,venta,2024-08-30 12:20:29.653326
1396,-33.40269,-70.576525,150.0,3.0,3.0,118.0,92.0,2.0,NaN,NaN,...,None,120000.0,Se Vende Departamento,https://www.portalinmobiliario.com/MLC-2653607...,metro_manquehue,1,380000000.0,10067.291898,venta,2024-08-30 12:20:29.660817
1397,-33.409007,-70.573441,16.0,3.0,2.0,87.0,82.0,1.0,1.0,19.0,...,NP,150000.0,"Oportunidad, Vendo Buen Departamento, Espectac...",https://www.portalinmobiliario.com/MLC-2656752...,metro_manquehue,1,301968000.0,8000.0,venta,2024-08-30 12:20:29.670020
1398,-33.40859,-70.57672,1.0,2.0,2.0,72.0,67.0,1.0,1.0,20.0,...,SO,NaN,Departamento Los Militares / La Gloria,https://www.portalinmobiliario.com/MLC-1520443...,metro_manquehue,1,263844540.0,6990.0,venta,2024-08-30 12:20:29.677459


# DATA PROCESING
------------------------------------------

In [8]:
# df_arriendo = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\metro_chile_españa\results_invesment_arriendo_departamento_1143_18_07_24.xlsx")

df_arriendo = df.query("tipo_operacion=='arriendo'")
len(df_arriendo)

255

In [16]:
df_arriendo.head()

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,orientacion,gastos_comunes,titulo,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date
0,-33.405653,-70.560099,2.0,2.0,2.0,188.0,68.0,2.0,1.0,7.0,...,Sur,180000.0,Con Jardín: Nuestra Señora Del Rosario / Estad...,https://www.portalinmobiliario.com/MLC-1519331...,metro_manquehue,0,1321110.0,35.0,arriendo,2024-08-30 10:19:34.222238
1,-33.410475,-70.5725,2.0,4.0,3.0,130.0,124.0,2.0,1.0,31.0,...,Nor-Oriente,0.0,Departamento En Arriendo De 4 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-1519330...,metro_manquehue,0,1509840.0,40.0,arriendo,2024-08-30 10:19:34.229592
2,-33.401758,-70.558145,23.0,2.0,2.0,70.0,63.0,0.0,1.0,0.0,...,Nor-Poniente,190000.0,Espectacular Departamento Tipo Mariposa En Las...,https://www.portalinmobiliario.com/MLC-1511659...,metro_manquehue,0,1019142.0,27.0,arriendo,2024-08-30 10:19:34.237387
3,-33.412626,-70.577014,23.0,1.0,1.0,38.0,38.0,1.0,1.0,4.0,...,Poniente,115000.0,Departamento Arriendo De 1 Dorm. Estadio Españ...,https://www.portalinmobiliario.com/MLC-2643503...,metro_manquehue,0,668859.12,17.72,arriendo,2024-08-30 10:19:34.245242
4,-33.411946,-70.566881,22.0,3.0,2.0,90.0,80.0,1.0,1.0,10.0,...,Poniente,240000.0,"Depto En Arriendo De 3 Dorm. En Las Condes, Me...",https://www.portalinmobiliario.com/MLC-1512054...,metro_manquehue,0,1050000.0,27.817517,arriendo,2024-08-30 10:19:34.252004


### precio arriendo segun tipologia

In [17]:
df_arriendo.groupby(by="n_dormitorios").mean()

c:\users\espdatascienceat\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3524: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Latitude,Longitude,dias_desde_publicacion,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,cantidad_pisos_edificio,piso_unidad,gastos_comunes,listed,Price,Price_UF
n_dormitorios,,,,,,,,,,,,,,,
1.0,-33.407880,-70.566138,29.512195,1.036585,46.921585,41.011707,1.524390,1.146341,9.085714,9.647059,37.915493,100950.05,0.000000,2.782647e+06,73.720315
2.0,-33.407733,-70.567749,20.309859,1.915493,78.271972,68.600423,1.028169,0.774648,88.865385,10.377778,6.452830,96652289.65,0.014085,4.668449e+06,123.680616
3.0,-33.407806,-70.570551,25.721519,2.455696,121.311519,108.354810,1.367089,0.835443,13.295082,11.953846,8.909091,218750.00,0.012658,1.448748e+06,38.381507
4.0,-33.405828,-70.569030,33.086957,3.043478,148.304348,134.434783,1.956522,0.956522,14.809524,10.450000,6.100000,275937.50,0.043478,1.782485e+06,47.223158


In [11]:
def remove_outliers(self, column_name):
    """ removedor de outliers para columna especificada"""
    Q1 = self[column_name].quantile(0.25)
    Q3 = self[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    condition = (self[column_name] >= lower_bound) & (self[column_name] <= upper_bound)
    return self.loc[condition]

# Attach the function to the pandas DataFrame class
pd.DataFrame.remove_outliers = remove_outliers

In [26]:
# EXTRACTOR DE PRECIO DE ARRIENDO SEGUN TIPOLOGIA
results_precio_arriendo = {}
for n_dorm in pd.Series(df_arriendo["n_dormitorios"].unique()).sort_values().values:

    clean_tipology_df = df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('Price_UF')
    precio_arriendo = int(clean_tipology_df.Price.quantile(0.1))
    results_precio_arriendo[n_dorm] = precio_arriendo
    print(f"con {n_dorm} dormitorios, se puede arrendar en aproximadamente {precio_arriendo}, analizado desde {len(clean_tipology_df)} propiedades")
    
    # list_results.append(df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('precio_UF'))
    
# df_price_wo_outliers = pd.concat(list_results)
# df_price_wo_outliers

con 1.0 dormitorios, se puede arrendar en aproximadamente 570000, analizado desde 73 propiedades
con 2.0 dormitorios, se puede arrendar en aproximadamente 766000, analizado desde 67 propiedades
con 3.0 dormitorios, se puede arrendar en aproximadamente 1049603, analizado desde 75 propiedades
con 4.0 dormitorios, se puede arrendar en aproximadamente 1509840, analizado desde 21 propiedades


In [27]:
results_precio_arriendo

{1.0: 570000, 2.0: 766000, 3.0: 1049603, 4.0: 1509840}

### precio por metro cuadrado

In [30]:
# EXTRACTOR DE PRECIO DE ARRIENDO SEGUN TIPOLOGIA
list_results = []
for n_dorm in pd.Series(df_arriendo["n_dormitorios"].unique()).sort_values().values:
    clean_tipology_df = df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('superficie_util')
    list_results.append(clean_tipology_df)

    # print(f"con {n_dorm} dormitorios, se puede arrendar en aproximadamente {precio_arriendo}, analizado desde {len(clean_tipology_df)} propiedades")
    # list_results.append(df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('precio_UF'))

df_price_wo_outliers = pd.concat(list_results)
df_price_wo_outliers["precio_m2"] = df_price_wo_outliers.Price_UF/df_price_wo_outliers.superficie_util
promedio_zona = int(np.mean(df_price_wo_outliers.Price/df_price_wo_outliers.superficie_util))
promedio_zona_UF = np.round(np.mean(df_price_wo_outliers.Price_UF/df_price_wo_outliers.superficie_util),2)

print(f"precio/m2 de la zona es {promedio_zona} CLP , {promedio_zona_UF} UF")

precio/m2 de la zona es 30305 CLP , 0.8 UF


In [31]:
df_price_wo_outliers

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,gastos_comunes,titulo,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date,precio_m2
3,-33.412626,-70.577014,23.0,1.0,1.0,38.0,38.0,1.0,1.0,4.0,...,115000.0,Departamento Arriendo De 1 Dorm. Estadio Españ...,https://www.portalinmobiliario.com/MLC-2643503...,metro_manquehue,0,668859.12,17.72,arriendo,2024-08-30 10:19:34.245242,0.466316
7,-33.408244,-70.563937,8.0,1.0,1.0,43.0,40.0,1.0,1.0,15.0,...,NaN,Depto En Apoquindo Metro Manquehue,https://www.portalinmobiliario.com/MLC-1516809...,metro_manquehue,0,650000.0,17.220368,arriendo,2024-08-30 10:19:34.273055,0.430509
10,-33.408351,-70.568972,10.0,1.0,1.0,60.0,48.0,1.0,1.0,20.0,...,NaN,Alonso De Córdova / Apoquindo,https://www.portalinmobiliario.com/MLC-1515933...,metro_manquehue,0,679428.0,18.0,arriendo,2024-08-30 10:19:34.296274,0.375
21,-33.407023,-70.557869,17.0,1.0,1.0,31.0,31.0,1.0,1.0,0.0,...,NaN,Excelente Dpto Amoblado Con Estacionamiento Y ...,https://www.portalinmobiliario.com/MLC-1513553...,metro_manquehue,0,700000.0,18.545011,arriendo,2024-08-30 10:19:34.379954,0.598226
24,-33.406685,-70.567951,40.0,1.0,1.0,57.0,42.0,1.0,0.0,0.0,...,NaN,"Excelente Departamento Cercano A Clínicas, Met...",https://www.portalinmobiliario.com/MLC-2601089...,metro_manquehue,0,1057642.92,28.02,arriendo,2024-08-30 10:19:34.402720,0.667143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,-33.40269,-70.576525,210.0,4.0,3.0,155.0,140.0,2.0,1.0,20.0,...,375000.0,Estupendo Departamento Con Vista Despejada Las...,https://www.portalinmobiliario.com/MLC-2655610...,metro_manquehue,0,1547586.0,41.0,arriendo,2024-08-30 10:19:35.771585,0.292857
205,-33.4032,-70.562362,34.0,4.0,3.0,170.0,145.0,2.0,1.0,16.0,...,500000.0,Estupendo Amplio Departamento A Pasos De Coleg...,https://www.portalinmobiliario.com/MLC-2616692...,metro_manquehue,0,1520000.0,40.269168,arriendo,2024-08-30 10:19:35.787745,0.277718
212,-33.407455,-70.570759,120.0,4.0,2.0,152.0,140.0,2.0,1.0,8.0,...,240000.0,Departamento En Arriendo Full Amoblado 4 Dorm....,https://www.portalinmobiliario.com/MLC-1509716...,metro_manquehue,0,2500000.0,66.232184,arriendo,2024-08-30 10:19:35.841283,0.473087
225,-33.408257,-70.57408,20.0,4.0,3.0,150.0,140.0,2.0,1.0,NaN,...,280000.0,Arriendo Amplio Departamento 4 Dormitorios La...,https://www.portalinmobiliario.com/MLC-2648918...,metro_manquehue,0,1700000.0,45.037885,arriendo,2024-08-30 10:19:35.940826,0.321699


#### CAPRATE BRUTO

In [32]:
# df_venta = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\metro_chile_españa\results_invesment_venta_departamento_2203_16_07_24.xlsx")

df_venta = df.query("tipo_operacion=='venta'")
len(df_venta)

1145

In [33]:
df_venta.head()

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,orientacion,gastos_comunes,titulo,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date
255,-33.405159,-70.567442,0.0,3.0,2.0,114.0,96.0,2.0,1.0,13.0,...,Norte,240000.0,"Depto 3 Dormitorios, 2 Baños, Barrio Noruega",https://www.portalinmobiliario.com/MLC-1502918...,metro_manquehue,1,358587000.0,9500.0,venta,2024-08-30 12:20:20.641181
256,-33.413052,-70.577703,0.0,4.0,3.0,170.0,150.0,2.0,1.0,26.0,...,Sur-Poniente,215000.0,Departamento En Venta De 4 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-1511026...,metro_manquehue,1,479374200.0,12700.0,venta,2024-08-30 12:20:20.649182
257,-33.409678,-70.568049,0.0,3.0,3.0,168.0,158.0,2.0,2.0,25.0,...,Nor-Oriente,250000.0,Departamento En Venta De 3 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-2439430...,metro_manquehue,1,464275800.0,12300.0,venta,2024-08-30 12:20:20.657084
258,-33.40552,-70.56707,0.0,3.0,3.0,145.0,111.0,2.0,1.0,14.0,...,Sur-Oriente,232000.0,Departamento En Venta De 3 Dorm Barrio Noruega...,https://www.portalinmobiliario.com/MLC-2437331...,metro_manquehue,1,422755200.0,11200.0,venta,2024-08-30 12:20:20.666077
259,-33.406566,-70.558591,0.0,2.0,2.0,91.0,76.0,2.0,1.0,13.0,...,Oriente,260000.0,Departamento En Venta De 2 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-2639441...,metro_manquehue,1,294418800.0,7800.0,venta,2024-08-30 12:20:20.674552


In [42]:
df_venta["caprate_bruto"]=np.NaN
for n_dorm in pd.Series(df_venta["n_dormitorios"].unique()).sort_values().values:
    if n_dorm in results_precio_arriendo.keys():
        df_venta.loc[df_venta["n_dormitorios"]==n_dorm, "caprate_bruto" ] = results_precio_arriendo[n_dorm] * 12 / df_venta.query(f"n_dormitorios=={n_dorm}").Price

df_venta.caprate_bruto = df_venta.caprate_bruto.astype(float)

c:\users\espdatascienceat\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\espdatascienceat\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\users\espdatascienceat\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexin

In [47]:
df_venta

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,gastos_comunes,titulo,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date,caprate_bruto
255,-33.405159,-70.567442,0.0,3.0,2.0,114.0,96.0,2.0,1.0,13.0,...,240000.0,"Depto 3 Dormitorios, 2 Baños, Barrio Noruega",https://www.portalinmobiliario.com/MLC-1502918...,metro_manquehue,1,358587000.0,9500.0,venta,2024-08-30 12:20:20.641181,0.035125
256,-33.413052,-70.577703,0.0,4.0,3.0,170.0,150.0,2.0,1.0,26.0,...,215000.0,Departamento En Venta De 4 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-1511026...,metro_manquehue,1,479374200.0,12700.0,venta,2024-08-30 12:20:20.649182,0.037795
257,-33.409678,-70.568049,0.0,3.0,3.0,168.0,158.0,2.0,2.0,25.0,...,250000.0,Departamento En Venta De 3 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-2439430...,metro_manquehue,1,464275800.0,12300.0,venta,2024-08-30 12:20:20.657084,0.027129
258,-33.40552,-70.56707,0.0,3.0,3.0,145.0,111.0,2.0,1.0,14.0,...,232000.0,Departamento En Venta De 3 Dorm Barrio Noruega...,https://www.portalinmobiliario.com/MLC-2437331...,metro_manquehue,1,422755200.0,11200.0,venta,2024-08-30 12:20:20.666077,0.029793
259,-33.406566,-70.558591,0.0,2.0,2.0,91.0,76.0,2.0,1.0,13.0,...,260000.0,Departamento En Venta De 2 Dorm. En Las Condes,https://www.portalinmobiliario.com/MLC-2639441...,metro_manquehue,1,294418800.0,7800.0,venta,2024-08-30 12:20:20.674552,0.031221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,-33.410506,-70.573783,7.0,4.0,3.0,115.0,115.0,1.0,1.0,45.0,...,220000.0,Dpto Remodelado Luminoso,https://www.portalinmobiliario.com/MLC-2670120...,metro_manquehue,1,320841000.0,8500.0,venta,2024-08-30 12:20:29.653326,0.056471
1396,-33.40269,-70.576525,150.0,3.0,3.0,118.0,92.0,2.0,NaN,NaN,...,120000.0,Se Vende Departamento,https://www.portalinmobiliario.com/MLC-2653607...,metro_manquehue,1,380000000.0,10067.291898,venta,2024-08-30 12:20:29.660817,0.033145
1397,-33.409007,-70.573441,16.0,3.0,2.0,87.0,82.0,1.0,1.0,19.0,...,150000.0,"Oportunidad, Vendo Buen Departamento, Espectac...",https://www.portalinmobiliario.com/MLC-2656752...,metro_manquehue,1,301968000.0,8000.0,venta,2024-08-30 12:20:29.670020,0.041710
1398,-33.40859,-70.57672,1.0,2.0,2.0,72.0,67.0,1.0,1.0,20.0,...,NaN,Departamento Los Militares / La Gloria,https://www.portalinmobiliario.com/MLC-1520443...,metro_manquehue,1,263844540.0,6990.0,venta,2024-08-30 12:20:29.677459,0.034839


In [54]:
df_venta.query("caprate_bruto >= .06 and Price < 180000000")

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,gastos_comunes,titulo,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date,caprate_bruto
566,-33.402368,-70.568001,56.0,3.0,1.0,60.0,60.0,0.0,0.0,50.0,...,20000.0,Rentable Depto 3d-1b Cerro El Plomo / Manquehu...,https://www.portalinmobiliario.com/MLC-2551855...,metro_manquehue,1,143057340.0,3790.0,venta,2024-08-30 12:20:23.066583,0.088043
735,-33.403952,-70.565967,60.0,3.0,1.0,70.0,70.0,0.0,NaN,NaN,...,50000.0,"Oportunidad Villa San Luis Las Condes 3d, 1b +...",https://www.portalinmobiliario.com/MLC-1496960...,metro_manquehue,1,139660200.0,3700.0,venta,2024-08-30 12:20:24.386305,0.090185


In [56]:
df_arriendo.query("n_dormitorios==3")

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,orientacion,gastos_comunes,titulo,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date
4,-33.411946,-70.566881,22.0,3.0,2.0,90.0,80.0,1.0,1.0,10.0,...,Poniente,240000.0,"Depto En Arriendo De 3 Dorm. En Las Condes, Me...",https://www.portalinmobiliario.com/MLC-1512054...,metro_manquehue,0,1050000.0,27.817517,arriendo,2024-08-30 10:19:34.252004
15,-33.406522,-70.570183,1.0,3.0,3.0,150.0,120.0,1.0,1.0,12.0,...,NOSP,NaN,Furnished- Modern- Nice Apartment Near Manque...,https://www.portalinmobiliario.com/MLC-2679146...,metro_manquehue,0,2150000.0,56.959678,arriendo,2024-08-30 10:19:34.333898
17,-33.411938,-70.569399,90.0,3.0,3.0,140.0,0.0,2.0,0.0,NaN,...,None,NaN,Se Arrienda Departamento En Las Condes!!!,https://www.portalinmobiliario.com/MLC-1507013...,metro_manquehue,0,1887300.0,50.0,arriendo,2024-08-30 10:19:34.350575
18,-33.402772,-70.56168,8.0,3.0,3.0,135.0,135.0,1.0,1.0,NaN,...,None,NaN,Excelente Depto Sercano Colegio Seck - Metro M...,https://www.portalinmobiliario.com/MLC-2668554...,metro_manquehue,0,1600000.0,42.388597,arriendo,2024-08-30 10:19:34.357876
19,-33.405539,-70.563052,180.0,3.0,3.0,113.0,96.0,1.0,1.0,NaN,...,SP,NaN,Amoblado 3d/3b - Sector Noruega - Metro Manque...,https://www.portalinmobiliario.com/MLC-1510518...,metro_manquehue,0,1600000.0,42.388597,arriendo,2024-08-30 10:19:34.365665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,-33.406922,-70.564282,6.0,3.0,3.0,174.0,174.0,2.0,0.0,NaN,...,None,300000.0,Arriendo Lujoso Departamento-noruega- Las Condes,https://www.portalinmobiliario.com/MLC-2671850...,metro_manquehue,0,2000000.0,52.985747,arriendo,2024-08-30 10:19:36.086008
248,-33.410917,-70.571443,9.0,3.0,2.0,110.0,100.0,1.0,1.0,NaN,...,NP,250000.0,Depto La Capitania,https://www.portalinmobiliario.com/MLC-1516826...,metro_manquehue,0,950000.0,25.16823,arriendo,2024-08-30 10:19:36.115769
249,-33.409537,-70.577296,17.0,3.0,3.0,110.0,110.0,2.0,1.0,NaN,...,NO,380000.0,"Depto 3 Dormitorios + 2 Estacionamientos, Apoq...",https://www.portalinmobiliario.com/MLC-2653214...,metro_manquehue,0,1170126.0,31.0,arriendo,2024-08-30 10:19:36.122611
250,-33.409798,-70.575207,21.0,3.0,2.0,110.0,110.0,1.0,1.0,18.0,...,NP,210000.0,Depto. Arriendo 3d+2b+ 1est+1bod Excelente Vis...,https://www.portalinmobiliario.com/MLC-1512501...,metro_manquehue,0,1132380.0,30.0,arriendo,2024-08-30 10:19:36.130409
